The goal for this piece of code is:
- Clean the data for the following datasets:
    - K2
    - APOKSAC
    - APOGEE
    - GALAH

After we have cleaned the datas is
- Use K2 model to predict the mass for APOGEE and GALAH
- Use APOKSAC model to predict mass for APOGEE

The goal is to get a table contain the following for all the data sets and use this to predict the mass of stars.
The table include parameters such as:

- Effective Temperature
- Log g
- Iron Abundance [Fe/H]
- Alpha Abundance [alpha/Fe]
- Carbon Abundance [C/Fe]
- Nitrogen Abundance [N/Fe]
- Oxygen Abundance [O/Fe]

Importing the required packages

In [8]:
from astropy.table import Table
import pandas as pd         # For table/data management

# Cleaning the GALAH Data

In [9]:
# Import the data and change into a data frame
galah = Table.read("Data/galah_dr4_allspec_220713.fits", format = "fits")
names = [name for name in galah.colnames if len(galah[name].shape) <= 1]
galah = galah[names].to_pandas()

For Apogee Data, there are a few recommendations (this usually indicates that the data has some issues, e.g. unreliably broadening and other issues) for the flag that should be applied:
- `snr_c3_iraf > 30`
- `flag_sp == 0`
- For any element `X`, the flag `flag_X_fe == 0`

More information can be found [the GALAH Survey Website](https://www.galah-survey.org/dr3/flags/)

In [10]:
# Filter the data
# galah = galah.loc[galah["snr_c3_iraf"] > 30]
galah = galah.loc[galah["flag_sp"] == 0]
galah = galah.loc[galah["flag_fe_h"] == 0]
galah = galah.loc[galah["flag_al_fe"] == 0]
galah = galah.loc[galah["flag_c_fe"] == 0]
galah = galah.loc[galah["flag_n_fe"] == 0]
galah = galah.loc[galah["flag_o_fe"] == 0]

Now the appropriate flag has been used, we extract only the useful columns and save the data for later.

In [11]:
galah = galah[["teff", "logg", "fe_h", "al_fe", "c_fe", "n_fe", "o_fe"]]
# Rmeove all rows with null values
galah.dropna(axis=0)
galah.to_csv("Data/cleaned_galah.csv", index=False)

# Cleaning APOGEE Data

In [16]:
apogee = Table.read("Data/allStar-dr17-synspec_rev1.fits", format="fits", hdu=1)
names = [name for name in apogee.colnames if len(apogee[name].shape) <= 1]
apogee = apogee[names].to_pandas()

For APOGEE, we do something similar by making sure all the parameters are working on are good.
More information can be found [SDSS explanantion of bitmasks](https://www.sdss.org/dr13/algorithms/bitmasks/#APOGEE_ASPCAPFLAG) and [SDSS recommendations for bitmasks](https://www.sdss.org/dr13/irspec/abundances/)

The parameters
- `ASPCAPFLAG & STAR_BAD == 0` (`STAR_BAD` is the 23rd bit (starting from 0), or  `STAR_BAD = 0b 1000 0000 0000 0000 0000 0000`)
    - `STAR_BAD` is a general indication of whether there are issues with measuring surface temperature, surface gravity, and check if any elements have `GRIDEDGE_BAD` set
- `ELEMFLAG`
    - Any example of this is `FE_H_FLAG`
        - Make sure that the flag does not contain `GRIDEDGE_BAD`, `GRIDEDGE_WARN` and `CALRANGE_BAD`
    - We should just check that the flag is equal to zero to eliminate all and any problematic behaviour

In [17]:
# Filter the data
STAR_BAD = 1 << 23
apogee = apogee.loc[apogee['ASPCAPFLAG'] & STAR_BAD == 0]
apogee = apogee.loc[apogee['FE_H_FLAG'] == 0]
apogee = apogee.loc[apogee['C_FE_FLAG'] == 0]
apogee = apogee.loc[apogee['AL_FE_FLAG'] == 0]
apogee = apogee.loc[apogee['N_FE_FLAG'] == 0]
apogee = apogee.loc[apogee['O_FE_FLAG'] == 0]

Extract the useful column and save for later

In [18]:
apogee = apogee[["TEFF", "LOGG", "FE_H", "AL_FE", "C_FE", "N_FE", "O_FE"]]
apogee.columns = apogee.columns.str.lower()
apogee.dropna(axis=0)
apogee.to_csv("Data/cleaned_apogee.csv", index=False)

# Cleaning APOKSAC 2 Data
TODO: Not sure how to do this


# Cleaning K2 Data
TODO: Not sure how to do this

In [31]:
k2_cols = ["EPIC", "kappa-r-c", "e_kappa_r-c", "kappa-m-c", "e_kappa-m-c", "kappa-r-A2Z", "kappa-r-c-BAM", "kappa-r-c-BHM", "kappa-r-c-CAN", "kappa-r-c-SYD", "e_kappa-r-c-A2Z", "e_kappa-r-c-BAM", "e_kappa-r-c-BHM", "e_kappa-r-c-CAN", "e_kappa-r-c-COR", "e_kappa-r-c-SYD", "kappa-m_A2Z", "kappa-m-c-BAM", "kappa-m-c-BHM", "kappa-m-c-CAN", "kappa-m-c-COR", "kappa-m-c-SYD", "e_kappa-m_A2Z", "e_kappa-m-c-BAM", "e_kappa-m-c-BHM", "e_kappa-m-c-CAN", "e_kappa-m-c-COR", "e_kappa-m-c-SYD"]

k2_col_width = [
    (1, 9), (11, 16), (18, 23), (25, 29), (18, 23),
    (25, 29), (31, 35), (37, 42), (44, 49), (51, 56),
    (58, 63), (65, 70), (72, 77), (79, 84), (86, 91),
    (93, 98), (100, 104), (106, 110), (112, 118), (120, 124),
    (126, 130), (137, 143), (145, 149), (151, 155), (157, 162),
    (164, 169), (171, 175), (177, 181), (183, 187), (189, 194)
]
# k2 = pd.read_fwf("Data/k2_mass_coefficient_data.txt", colspecs=k2_col_width)
k2 = pd.read_fwf(
    "Data/k2_mass_coefficients.txt",
    skiprows=50,
    header=None,
    index_col=False,
    names = k2_cols,
    colspecs = k2_col_width
)
# k2.columns = k2_cols
k2

/home/roy/.local/lib/python3.9/site-packages/pandas/io/parsers/readers.py:762: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return _read(filepath_or_buffer, kwds)


,EPIC,kappa-r-c,e_kappa_r-c,kappa-m-c,e_kappa-m-c,kappa-r-A2Z,kappa-r-c-BAM,kappa-r-c-BHM,kappa-r-c-CAN,kappa-r-c-SYD,...,kappa-m-c-BHM,kappa-m-c-CAN,kappa-m-c-COR,kappa-m-c-SYD,e_kappa-m_A2Z,e_kappa-m-c-BAM,e_kappa-m-c-BHM,e_kappa-m-c-CAN,e_kappa-m-c-COR,e_kappa-m-c-SYD
0,1051625,4.757,2.215,0.324,2.215,0.324,0.404,7.176,NaN,6.728,...,NaN,0.366,NaN,1.034,0.645,NaN,1.131,NaN,0.616,0.160
1,1079541,0.610,0.316,0.523,0.316,0.523,0.102,3.809,1.093,NaN,...,NaN,0.552,0.608,1.555,0.563,NaN,0.341,0.113,NaN,0.105
2,1082897,9.833,0.440,0.027,0.440,0.027,0.095,2.600,9.651,0.477,...,0.381,0.654,0.976,0.975,0.065,0.089,0.259,0.130,0.238,0.075
3,1084449,2.473,0.440,0.544,0.440,0.544,0.123,4.888,2.275,3.176,...,NaN,0.196,0.446,NaN,0.657,NaN,0.560,0.098,0.481,NaN
4,1084664,4.002,1.691,0.076,1.691,0.076,0.272,2.506,NaN,NaN,...,NaN,0.788,NaN,1.230,0.006,NaN,0.866,NaN,NaN,0.161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18817,51619708,9.697,0.377,0.880,0.377,0.880,0.076,9.116,NaN,9.791,...,NaN,0.385,NaN,1.037,0.845,NaN,0.815,NaN,0.111,0.120
18818,51620873,9.329,0.471,0.663,0.471,0.663,0.102,NaN,NaN,9.734,...,NaN,NaN,NaN,NaN,0.674,NaN,NaN,NaN,0.164,NaN
18819,51621333,6.843,0.123,0.193,0.123,0.193,0.046,NaN,NaN,7.134,...,NaN,NaN,NaN,NaN,0.241,NaN,NaN,NaN,0.134,NaN
18820,51622742,1.998,0.927,0.497,0.927,0.497,0.248,2.233,NaN,3.303,...,0.602,0.561,NaN,1.180,0.680,0.674,0.265,NaN,0.426,0.284
